Implementación distribuida y paralelizada de un algoritmo de Machine Learning

In [8]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

import numpy as np
from pyspark import RDD
from typing import Tuple 


def readFile(filename):
    def transform_to_floats(line:str) -> np.ndarray:
        # Importante. Retornar como tupla
        element = [float(x) for x in line.split(',')]
        return (np.array(element))
    
    data_raw = contex_global.textFile(filename)
    return data_raw.map(transform_to_floats)


def _calculate_mean(rdd:RDD, total_rows:int) -> np.ndarray:
    def reduce_function(value1, value2):
        return value1[:11] + value2[:11]
    
    return rdd.reduce(reduce_function) / total_rows

def _calculate_stdev(rdd:RDD, mean:np.ndarray, total_rows:int) -> np.ndarray:
    def map_function(element):
        return (element[:11] - mean)**2

    def reduce_function(value1, value2):
        return value1[:11] + value2[:11]
    
    rdd_aux = rdd.map(map_function)
    return (rdd_aux.reduce(reduce_function) / total_rows)**(1/2)


def normalize(RDD_Xy:RDD):
    def map_function(element, mean, desvest):
        x_values = (element[:11] - mean) / desvest
        return np.concatenate((x_values, [element[-1]]))

    m = RDD_Xy.count()
    mean = _calculate_mean(RDD_Xy, m)
    desvest = _calculate_stdev(RDD_Xy, mean, m)
    return RDD_Xy.map(lambda element: map_function(element, mean, desvest))


def _model(W,b,X):    
    pass

def _sigmoid(z):
    return 1 / (1 + np.exp(-z))

def _cost_function_J(label, prediction):
    m = label.shape[0] # No tiene sentido calcular esto cada vez.
    return -np.sum(label * np.log(prediction) \
                   + (1 - label) * np.log(1 - prediction)) / m

def _convert_to_y_pred(value, threshold):
    return np.where(value > threshold, 1, 0)

def predict(W,b,x):
    pass

def accuracy(W, b, RDD_Xy:RDD):
    pass

def train(RDD_Xy:RDD, iterations, learning_rate):
    pass

#FILE_NAME = '../../0-SPAI/1-datos/botnet_tot_syn_l.csv'
FILE_NAME = 'botnet_sample.csv'
LEARNING_RATE = 1.5
N_ITER = 10

session_global = SparkSession.builder.master('local[*]').getOrCreate()
contex_global = session_global.sparkContext

# TODO
# read data
data_raw_rdd = readFile(FILE_NAME)

#data_raw_rdd.take(5)
#m = data_raw_rdd.count()
#media = _calculate_mean(data_raw_rdd, m)
#print('Media', media)
#desvest = _calculate_stdev(data_raw_rdd, media, m)
#print('Desvest = ', desvest)

# standarize
normal_data = normalize(data_raw_rdd)
normal_data.take(2)

## train
#ws = train(normal_data, N_ITER, LEARNING_RATE)

#acc = accuracy(ws, normal_data)
#print('Accuracy = ', acc)


[array([-0.79240977, -0.81309371, -0.42245076, -0.46646975, -0.52239296,
        -0.35631957,  0.7370103 ,  0.52834963,  0.82717799,  0.47316616,
         0.15895172,  1.        ]),
 array([ 1.4437204 ,  1.116361  ,  0.39127564, -0.46646975, -0.5223921 ,
        -0.35631927,  0.92633194,  1.48012075,  1.02641833, -0.82869545,
         0.15894853,  0.        ])]